In [ ]:
# =========================
# PART 1 — Imports & Sender Tools (Resend)
# =========================
from dotenv import load_dotenv
from typing import Dict, Optional
from agents import Agent, Runner, trace, function_tool
import os, re, html, asyncio
import resend

# Load .env (API keys, etc.)
load_dotenv(override=True)

# ---- Company context ----
COMPANY_NAME = "All American School of English Teaching"
COMPANY_EMAIL = "onboarding@resend.dev"
TIKTOK_HANDLE = "@all.american.eng"
INSTAGRAM_HANDLE = "all.american.eng"
TIKTOK_URL = "https://www.tiktok.com/@all.american.eng"
INSTAGRAM_URL = "https://www.instagram.com/all.american.eng"

# Keep BOTH to avoid NameError across parts
PRICE_PLN = int(os.getenv("PRICE_PLN", "120"))
PRICE_USD = float(os.getenv("PRICE_USD", "25"))

OFFERS = ["General English", "Business English", "Technology English", "Matura prep", "TOEFL"]

# Default subject for fast subject tool / fallback
os.environ.setdefault("RESEND_SUBJECT", "Want to speak native-level American English?")
os.environ.setdefault("REPLY_TO", COMPANY_EMAIL)  # where replies should go

# ---------- Utilities ----------
def _html_to_text_fallback(html_body: str) -> str:
    """Light HTML→text fallback to improve deliverability."""
    txt = html.unescape(html_body)
    txt = re.sub(r'(?i)<br\s*/?>', '\n', txt)
    txt = re.sub(r'(?i)</p\s*>', '\n\n', txt)
    txt = re.sub(
        r'(?is)<a\s+[^>]*href=["\']([^"\']+)["\'][^>]*>(.*?)</a>',
        lambda m: f"{re.sub('<[^>]+>', '', m.group(2)).strip()} ({m.group(1)})",
        txt
    )
    txt = re.sub(r'<[^>]+>', '', txt)
    txt = re.sub(r'\n{3,}', '\n\n', txt).strip()
    return txt

# ---------- FAST SUBJECT + HTML TOOLS (function_tool) ----------
import html as _html
import re as _re

def _paragraphize(text: str) -> str:
    """Turn plain text into simple <p> blocks, preserving blank lines."""
    safe = _html.escape(text.strip())
    paras = [p.strip().replace("\n", "<br>") for p in _re.split(r"\n\s*\n+", safe) if p.strip()]
    if not paras:
        return "<p></p>"
    return "\n".join(
        f"<p style='margin:0 0 12px 0; line-height:1.55; font-size:16px; color:#111;'>{p}</p>"
        for p in paras
    )

def _build_html_email(body_text: str) -> str:
    """Minimal, mobile-friendly HTML email with branding, CTA, offer block, and socials."""
    body_html = _paragraphize(body_text)
    return f"""<!doctype html>
<html>
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width,initial-scale=1">
  <title>{_html.escape(COMPANY_NAME)}</title>
</head>
<body style="margin:0;padding:0;background:#f6f7fb;font-family:-apple-system,BlinkMacSystemFont,'Segoe UI',Roboto,Helvetica,Arial,sans-serif;">
  <table role="presentation" width="100%" cellspacing="0" cellpadding="0" border="0">
    <tr>
      <td align="center" style="padding:24px 12px;">
        <table role="presentation" width="600" cellspacing="0" cellpadding="0" border="0" style="max-width:100%;background:#fff;border-radius:12px;overflow:hidden;box-shadow:0 4px 18px rgba(0,0,0,0.06);">
          <tr>
            <td style="background:#0ea5e9;color:#fff;padding:14px 20px;font-weight:600;font-size:16px;">
              {COMPANY_NAME}
            </td>
          </tr>
          <tr>
            <td style="padding:20px 24px;">
              {body_html}
            </td>
          </tr>
          <tr>
            <td style="padding:0 24px 16px 24px;">
              <a href="mailto:{COMPANY_EMAIL}" target="_blank"
                 style="display:inline-block;background:#0ea5e9;color:#fff;text-decoration:none;padding:12px 18px;border-radius:10px;font-weight:600;">
                 Book a lesson
              </a>
            </td>
          </tr>
          <tr>
            <td style="padding:0 24px 20px 24px;font-size:14px;color:#444;line-height:1.55;">
              <strong>What we offer</strong>
              <ul style="margin:8px 0 0 18px;padding:0;">
                {"".join(f"<li>{_html.escape(o)}</li>" for o in OFFERS)}
              </ul>
              <p style="margin:8px 0 0 0;"><strong>Price:</strong> {PRICE_USD} USD per hour</p>
            </td>
          </tr>
          <tr>
            <td style="padding:0 24px 20px 24px;font-size:13px;color:#555;">
              Contact: <a href="mailto:{COMPANY_EMAIL}">{COMPANY_EMAIL}</a><br>
              TikTok: <a href="{TIKTOK_URL}">{TIKTOK_HANDLE}</a> &nbsp;•&nbsp;
              Instagram: <a href="{INSTAGRAM_URL}">{INSTAGRAM_HANDLE}</a>
            </td>
          </tr>
        </table>
        <div style="max-width:600px;width:100%;font-size:12px;color:#6b7280;margin-top:12px;line-height:1.4;">
          You received this because you might be interested in English lessons. If not, please ignore.
        </div>
      </td>
    </tr>
  </table>
</body>
</html>"""

@function_tool
def subject_writer(message: str) -> str:
    """Fast subject line: use env default or fallback."""
    return os.environ.get("RESEND_SUBJECT", "Want to speak native-level American English?")

@function_tool
def html_converter(body: str) -> str:
    """Fast HTML converter using the fixed template (no LLM call)."""
    return _build_html_email(body)

# ---------- Standard sender used by Email Manager ----------
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """
    Send HTML email via Resend to the default recipient (RESEND_TO).
    Includes reply_to so responses go to your inbox.
    """
    api_key = os.environ.get("RESEND_API_KEY")
    if not api_key:
        return {"status": "failure", "message": "RESEND_API_KEY not set in environment."}
    resend.api_key = api_key

    from_email = os.environ.get("RESEND_FROM", COMPANY_EMAIL)  # must be verified in Resend
    to_email   = os.environ.get("RESEND_TO", "miketayloriii15@gmail.com")
    subject    = (subject or "").strip() or os.environ.get("RESEND_SUBJECT", "English lessons")

    text_body = _html_to_text_fallback(html_body)

    try:
        resp = resend.Emails.send({
            "from": from_email,
            "to": [to_email],
            "subject": subject,
            "text": text_body,
            "html": html_body,
            "reply_to": os.environ.get("REPLY_TO", COMPANY_EMAIL),
        })
        return {"status": "success", "id": (resp or {}).get("id"), "to": to_email, "from": from_email, "subject": subject}
    except Exception as e:
        r = getattr(e, "response", None)
        return {"status": "failure", "message": repr(e), "status_code": getattr(r, "status_code", None), "response": getattr(r, "text", None)}

# ---------- Dynamic sender for replies (variable recipient + threading) ----------
@function_tool
def send_html_email_dynamic(
    to_email: str,
    subject: str,
    html_body: str,
    in_reply_to: Optional[str] = None,
    references: Optional[str] = None,
) -> Dict[str, str]:
    """Send HTML email via Resend to an arbitrary recipient, preserving thread headers."""
    api_key = os.environ.get("RESEND_API_KEY")
    if not api_key:
        return {"status": "failure", "message": "RESEND_API_KEY not set in environment."}
    resend.api_key = api_key

    from_email = os.environ.get("RESEND_FROM", COMPANY_EMAIL)  # verified sender
    payload = {
        "from": from_email,
        "to": [to_email],
        "subject": (subject or "").strip() or os.environ.get("RESEND_SUBJECT", "Re: English lessons"),
        "text": _html_to_text_fallback(html_body),
        "html": html_body,
        "reply_to": os.environ.get("REPLY_TO", COMPANY_EMAIL),
    }

    headers = {}
    if in_reply_to:
        headers["In-Reply-To"] = in_reply_to
    if references:
        headers["References"] = references
    if headers:
        payload["headers"] = headers  # Resend supports custom headers

    try:
        resp = resend.Emails.send(payload)
        return {"status": "success", "id": (resp or {}).get("id"), "to": to_email}
    except Exception as e:
        r = getattr(e, "response", None)
        return {"status": "failure", "message": repr(e), "status_code": getattr(r, "status_code", None), "response": getattr(r, "text", None)}



In [3]:
# =========================
# PART 2 — Tools & Email Manager
# =========================

# NOTE: subject_writer and html_converter are FAST function tools from PART 1.
# We DO NOT redefine them here (avoids extra LLM calls and name collisions).

# ---- Three sales agents (draft generators) ----
benefits_style = f"""
You are a cold email writer for {COMPANY_NAME}. Write a concise, plain-text draft (no HTML, no subject).
Audience: potential students/parents in Poland. Offers: {", ".join(OFFERS)} at {PRICE_USD} USD/hour.
Style: benefit-led (outcomes, confidence, results). 90–140 words. Start with a natural greeting ("Dear {{first_name}}" if provided, else "Hello").
Include one clear CTA to reply or book a trial (mailto:{COMPANY_EMAIL}). Avoid buzzwords; be specific.
"""

social_proof_style = f"""
You are a cold email writer for {COMPANY_NAME}. Plain-text, no subject, no HTML.
Emphasize credibility: TESOL certification, successful student outcomes (e.g., Matura/TOEFL improvements), and 1:1 tailored lessons.
Include offers: {", ".join(OFFERS)} at {PRICE_USD} USD/hour. 90–140 words. One CTA to email {COMPANY_EMAIL}.
"""

concise_style = f"""
You are a cold email writer for {COMPANY_NAME}. Keep it ultra concise (60–90 words), plain-text, no subject, no HTML.
Direct value prop + simple CTA ("Reply 'trial' to book a 30-min session" or email {COMPANY_EMAIL}).
Mention a specific offer only if context hints (Business, Technology, Matura, TOEFL).
"""

sales_agent1 = Agent(name="Sales Agent — Benefits", instructions=benefits_style, model="gpt-4o-mini")
sales_agent2 = Agent(name="Sales Agent — Social Proof", instructions=social_proof_style, model="gpt-4o-mini")
sales_agent3 = Agent(name="Sales Agent — Concise", instructions=concise_style, model="gpt-4o-mini")

sales_tools = [
    sales_agent1.as_tool(tool_name="sales_agent1", tool_description="Benefit-led cold email draft"),
    sales_agent2.as_tool(tool_name="sales_agent2", tool_description="Social-proof cold email draft"),
    sales_agent3.as_tool(tool_name="sales_agent3", tool_description="Concise cold email draft"),
]

# ---- Email pipeline tools for the Email Manager (use fast function tools from Part 1) ----
email_tools = [subject_writer, html_converter, send_html_email]

# ---- Email Manager (formats + sends) ----
email_manager_instructions = (
    "You are an email formatter and sender. You receive the body of an email to be sent. "
    "Step 1: Call subject_writer ONCE to write a subject. "
    "Step 2: Call html_converter ONCE to convert the body to a complete HTML email. "
    "Step 3: Call send_html_email with the subject and HTML body. "
    "After Step 3, OUTPUT exactly a one-line confirmation with the send_html_email 'id' and STOP. "
    "Do not call any more tools after sending. Do not ask follow-up questions."
)

emailer_agent = Agent(
    name="Email Manager",
    instructions=email_manager_instructions,
    tools=email_tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it"
)

handoffs = [emailer_agent]

# Optional debug
# print("Tools:", sales_tools + email_tools)
# print("Handoffs:", handoffs)

# ---- Reply writer (plain text → short, brand-safe reply) ----
reply_writer = Agent(
    name="Email Reply Writer",
    model="gpt-4o-mini",
    instructions=f"""
You are a concise email reply assistant for {COMPANY_NAME}.
Pricing: {PRICE_USD} USD/hour. Offers: {", ".join(OFFERS)}.
Tone: warm, clear, professional. ≤ 120 words unless multiple questions.
Tasks:
- Answer their questions directly.
- Offer a 15–30 minute trial or propose 2–3 time slots (Europe/Warsaw).
- CTA: reply or email {COMPANY_EMAIL}.
Language: English.
Output plain text only (no HTML).
"""
)




In [4]:
# =========================
# PART 3 — Sales Manager & Run
# =========================

sales_manager_instructions = f"""
You are a Sales Manager at {COMPANY_NAME}.
Your goal is to find the single best cold sales email using the sales_agent tools for learners/parents in Poland.
Offers: {", ".join(OFFERS)} at {PRICE_USD} USD/hour.

Process:
1) Generate Drafts:
   - Call ALL THREE tools: sales_agent1, sales_agent2, sales_agent3 to produce three distinct PLAIN-TEXT drafts (no HTML, no subject).
2) Evaluate and Select:
   - Choose ONE winning draft based on clarity, relevance (Polish vs English as appropriate),
     specific value (e.g., Matura/Business/TOEFL when context suggests), and a clear CTA to reply or book a trial.
3) Handoff for Sending:
   - Hand off EXACTLY ONE winning draft (the plain-text body only) to the 'Email Manager' agent.

Termination rule:
- After handing off to the Email Manager, OUTPUT a short line confirming the handoff and STOP.
- Do NOT call any more tools after the handoff. Do NOT ask follow-up questions.
"""

sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=sales_tools,     # from Part 2: [sales_agent1, sales_agent2, sales_agent3] as tools
    handoffs=handoffs,     # from Part 2: [emailer_agent]
    model="gpt-4o-mini",
)

# --- Run (Notebook/IPython safe) ---
async def main():
    # You can change this message to whatever kick-off instruction you want
    message = "Write a cold email for a potential student in Warsaw; greet with 'Dear Parent' and sign from Mike."
    with trace("Automated SDR"):
        # Give enough turns for: 3 drafts -> select -> handoff -> subject -> html -> send -> stop
        result = await Runner.run(sales_manager, message, max_turns=25)
    print(result)

# In a notebook, call with top-level await:
await main()

# If running as a plain .py script instead of a notebook, use:
# if __name__ == "__main__":
#     import asyncio
#     asyncio.run(main())


RunResult:
- Last agent: Agent(name="Email Manager", ...)
- Final output (str):
    The email has been sent successfully with ID: `send_html_email` and status: Unknown error.
- 33 new item(s)
- 12 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)
